In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
test = True

In [3]:
# load dataset
df = pd.read_csv('../data/dataset_clean.csv')
df.head()

,order_id,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_1_count,category_2_count,category_3_count,category_4_count,category_5_count,category_6_count,category_7_count,category_8_count,category_9_count,category_10_count,category_11_count,category_12_count,category_13_count,category_14_count,category_15_count,category_16_count,category_17_count,category_18_count,category_19_count,category_20_count,category_21_count,order_dow,order_hour_of_day
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,4,10
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,9
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5,17
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1,9
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0,6,16


In [4]:
if test:
    _, df, _, _ = train_test_split(df, df['category_1'],
                                   test_size=0.05,
                                   random_state=0,
                                   stratify=df[['order_dow', 'order_hour_of_day']])

In [5]:
# dow one hot
enc = OneHotEncoder()
df[[f"order_dow_{x}" for x in range(7)]] = enc.fit_transform(df.loc[:, 'order_dow'].values.reshape((-1, 1))).toarray()

# hod one hot
enc = OneHotEncoder()
df[[f"order_hod_{x}" for x in range(24)]] = enc.fit_transform(df.loc[:, 'order_hour_of_day'].values.reshape((-1, 1))).toarray()
df.head()

,order_id,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_1_count,category_2_count,category_3_count,category_4_count,category_5_count,category_6_count,category_7_count,category_8_count,category_9_count,category_10_count,category_11_count,category_12_count,category_13_count,category_14_count,category_15_count,category_16_count,category_17_count,category_18_count,category_19_count,category_20_count,category_21_count,order_dow,order_hour_of_day,order_dow_0,order_dow_1,order_dow_2,order_dow_3,order_dow_4,order_dow_5,order_dow_6,order_hod_0,order_hod_1,order_hod_2,order_hod_3,order_hod_4,order_hod_5,order_hod_6,order_hod_7,order_hod_8,order_hod_9,order_hod_10,order_hod_11,order_hod_12,order_hod_13,order_hod_14,order_hod_15,order_hod_16,order_hod_17,order_hod_18,order_hod_19,order_hod_20,order_hod_21,order_hod_22,order_hod_23
1290121,1319024,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,3.0,0.0,0.0,0,19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
340938,348662,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2335325,2387661,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,6,10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2723061,2784127,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6,16,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2353309,2406032,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,1.0,0.0,1,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
def partition_dataset(df,
                      label_col,
                      features=None,
                      partition_on=None,
                      test_size=0.2,
                      valid_size=0.2,
                      random_state=None,
                      three_way=False
                     ):
    if features is None:
        features = [col for col in df.columns if col != label_col]
    X = df[features]
    y = df[label_col].apply(int).values.reshape((-1, 1))
    if partition_on is not None:
        partition_on = df[partition_on]
        
    if three_way:
        rng = np.random.default_rng(random_state)
        seeds = rng.integers(10000, size=2)
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=seeds[0], 
                                                        stratify=partition_on
                                                       )
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                              test_size=test_size, 
                                                              random_state=seeds[1], 
                                                              stratify=partition_on
                                                             )
        return X_train, X_valid, X_test, y_train, y_valid, y_test
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=test_size, 
                                                            random_state=random_state, 
                                                            stratify=partition_on
                                                           )
        return X_train, X_test, y_train, y_test

In [7]:
# train test split stratified by label & dow
def train_xgb_cv_classifier(X, y,
                            test_size=0.2,
                            n_folds=5,
                            xgb_params={},
                            balanced_weight=False,
                            random_state=None
                           ):
    
    # handling xgb params and the defaults
    xgb_params_complete = {'n_estimators': 1000,
                           'learning_rate': 0.01,
                           'colsample_bytree': 0.8,
                           'subsample': 0.8}
    for key, val in xgb_params.items():
        xgb_params_complete[key] = val
    
    scores = []
    # CV fold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for train_index, valid_index in skf.split(X, y):
        X_train, X_valid = X.iloc[train_index, :], X.iloc[valid_index, :]
        y_train, y_valid = y[train_index], y[valid_index]
        
        # xgb training
        if balanced_weight:
            scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
        else:
            scale_pos_weight = None
        model = XGBClassifier(**xgb_params_complete, 
                              use_label_encoder=False,
                              scale_pos_weight=scale_pos_weight
                             )
        model.fit(X_train, y_train)
        
        # prediction on validation set
        pred = model.predict(X_valid)
        scores.append(recall_score(y_valid, pred))
        
    return np.array(scores).mean()

# Features trials

In [8]:
labels = [f"category_{x}" for x in [4, 16, 7, 19, 1]]
col_dow = [f"order_dow_{x}" for x in range(7)]
col_hod = [f"order_hod_{x}" for x in range(24)]

In [9]:
# TEST with indicator variables
labels = [f"category_{x}" for x in [4, 16, 7, 19, 1]]
col_dow = [f"order_dow_{x}" for x in range(7)]
scores_all_labels = []
for label in labels:
    print(label)
    X_train, X_test, y_train, y_test = partition_dataset(df, label, random_state=0)
    scores_all_labels.append(
        train_xgb_cv_classifier(X_train[[f"category_{x}" for x in range(1, 22) if f"category_{x}" != label]
                                        + [f"category_{x}_count" for x in range(1, 22) if f"category_{x}" != label]
                                        + col_dow
                                        + col_hod
                                       ], 
                                y_train,
                                balanced_weight=True,
                                xgb_params={'eval_metric': 'logloss'},
                                random_state=0
                               )
    )
    
pd.Series(scores_all_labels, index=[4, 16, 7, 19, 1])

category_4
category_16
category_7
category_19
category_1


4     0.633069
16    0.628663
7     0.551084
19    0.568203
1     0.600658
dtype: float64

In [10]:
scores_all_labels

[0.6330689415802601,
 0.6286629323664416,
 0.5510837438423646,
 0.5682033894024633,
 0.6006576425540466]

## Learning rate and n_estimators

#### complete features (indicator + count + dow + hod)

In [44]:
label = 'category_4'
df_xgb = df[[f"category_{x}" for x in range(1, 22)]
            + [f"category_{x}_count" for x in range(1, 22)]
            + col_dow
            + col_hod
           ]
X_train_, X_test, y_train_, y_test = partition_dataset(df_xgb, 
                                                       label, 
                                                       features=[x for x in df_xgb.columns if x not in [label, label+'_count']],
                                                       three_way=False,
                                                       random_state=0)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, valid_index in skf.split(X_train_, y_train_):
    X_train, X_valid = X_train_.iloc[train_index, :], X_train_.iloc[valid_index, :]
    y_train, y_valid = y_train_[train_index], y_train_[valid_index]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    watchlist = [(dtrain,'train'), (dvalid,'valid')]

    model = xgb.train({'learning_rate': 0.01,
                       'colsample_bytree': 0.8,
                       'subsample': 0.8,
                       'objective': 'binary:logistic'},
                      dtrain,
                      num_boost_round=5000,
                      evals=watchlist,
                      early_stopping_rounds=20,
                      verbose_eval=200)
    pred = model.predict(dvalid)
    pred = (pred > 0.5).astype(int)
    
    print(f"recall = {recall_score(y_valid, pred)}")
    print(f"f1 = {f1_score(y_valid, pred)}")
    print(f"accuracy = {accuracy_score(y_valid, pred)}")

[12:28:41] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.68980	valid-logloss:0.68982
[200]	train-logloss:0.50097	valid-logloss:0.50646
[400]	train-logloss:0.48569	valid-logloss:0.49525
[600]	train-logloss:0.48055	valid-logloss:0.49339
[800]	train-logloss:0.47736	valid-logloss:0.49287
[910]	train-logloss:0.47593	valid-logloss:0.49271
recall = 0.9598445595854922
f1 = 0.8647799447898198
accuracy = 0.7749262206283387
[12:29:41] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.68978	valid-logloss:0.68983
[200]	train-logloss:0.50068	valid-logloss:0.50672
[400]	train-loglos

#### indicator var

In [45]:
label = 'category_4'
df_xgb = df[[f"category_{x}" for x in range(1, 22)]
#             + [f"category_{x}_count" for x in range(1, 22)]
#             + col_dow
#             + col_hod
           ]
X_train_, X_test, y_train_, y_test = partition_dataset(df_xgb, 
                                                       label, 
                                                       features=[x for x in df_xgb.columns if x not in [label, label+'_count']],
                                                       three_way=False,
                                                       random_state=0)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, valid_index in skf.split(X_train_, y_train_):
    X_train, X_valid = X_train_.iloc[train_index, :], X_train_.iloc[valid_index, :]
    y_train, y_valid = y_train_[train_index], y_train_[valid_index]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    watchlist = [(dtrain,'train'), (dvalid,'valid')]

    model = xgb.train({'learning_rate': 0.01,
                       'colsample_bytree': 0.8,
                       'subsample': 0.8,
                       'objective': 'binary:logistic'},
                      dtrain,
                      num_boost_round=5000,
                      evals=watchlist,
                      early_stopping_rounds=20,
                      verbose_eval=200)
    pred = model.predict(dvalid)
    pred = (pred > 0.5).astype(int)
    
    print(f"recall = {recall_score(y_valid, pred)}")
    print(f"f1 = {f1_score(y_valid, pred)}")
    print(f"accuracy = {accuracy_score(y_valid, pred)}")

[12:36:20] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.69010	valid-logloss:0.69015
[200]	train-logloss:0.50419	valid-logloss:0.50806
[400]	train-logloss:0.49233	valid-logloss:0.49855
[600]	train-logloss:0.48931	valid-logloss:0.49732
[800]	train-logloss:0.48716	valid-logloss:0.49673
[973]	train-logloss:0.48542	valid-logloss:0.49632
recall = 0.9565065763252292
f1 = 0.8642358766599145
accuracy = 0.7746647241211849
[12:36:55] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.69015	valid-logloss:0.69014
[200]	train-logloss:0.50372	valid-logloss:0.50846
[400]	train-loglos